In [8]:
import os 

In [3]:
%pwd

'/Users/sunnyg/Desktop/End-to-end-Kidney_AI/research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'/Users/sunnyg/Desktop/End-to-end-Kidney_AI'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    """
    Data Ingestion Configuration
    """
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path



In [30]:
from CNN_Classifier.constants import *
# import everything
from CNN_Classifier.utils.common import read_yaml, create_directories   

In [ ]:

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = Path("config/config.yaml"),
        params_filepath = Path("params.yaml")):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config


In [47]:
import os 
import zipfile
import gdown
from CNN_Classifier import logger
from CNN_Classifier.utils.common import get_size

In [50]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [51]:
# pipeline is first step: download_file, and second step: extract_zip_file
# define the pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-03-23 18:46:59,335: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-03-23 18:46:59,338: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-23 18:46:59,339: INFO: common: created directory at: artifacts]
[2025-03-23 18:46:59,341: INFO: common: created directory at: artifacts/data_ingestion]
[2025-03-23 18:46:59,342: INFO: 391976878: Downloading data from https://drive.google.com/file/d/1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3&confirm=t&uuid=7d4d3249-7b54-4c7e-8ffe-fa9ed11ceb39
To: /Users/sunnyg/Desktop/End-to-end-Kidney_AI/artifacts/data_ingestion/data.zip
100%|██████████| 57.7M/57.7M [00:02<00:00, 25.7MB/s]

[2025-03-23 18:47:05,830: INFO: 391976878: Downloaded data from https://drive.google.com/file/d/1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3/view?usp=sharing into file artifacts/data_ingestion/data.zip]
